In [10]:
!pip install google-search-results

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32083 sha256=67fadc2539bba33ea7f966c4fca92d246beabbb6ce2c6b63220fdf752c40a291
  Stored in directory: c:\users\sebaa\appdata\local\pip\cache\wheels\0c\47\f5\89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import openai
api_key = #Aqui va el OpenAi api key
import chromadb
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

client = chromadb.PersistentClient(path=".")
collection = client.get_or_create_collection(
    name="hidrogeno_verde",
    metadata={"dimension": 1536}
)

C:\Users\sebaa\AppData\Local\Temp\ipykernel_13960\2279236151.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [2]:
import pandas as pd
df = pd.read_csv('hidrogeno_verde.csv')
# Convertir la columna 'date' al tipo datetime
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
# Formatear la columna 'date' al formato YYYY-MM-DD
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['combined'] = df['title'] + ' ' + df['text']
df

,date,id_news,text,title,url,polarity_score,polarity_label,media_outlet,country,combined
0,2024-10-19,54.272.642,"Santiago, octubre de 2024 – Como parte de la c...",Ausenco impulsa la transición energética a tra...,https://tierramarillano.cl/2024/10/20/ausenco-...,-,-,tierraamarillano,chile,Ausenco impulsa la transición energética a tra...
1,2024-10-17,54.270.980,"Claudio Seebach, una voz respetada en el sect...","Claudio Seebach aterriza la ""realidad"" del hid...",https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile,"Claudio Seebach aterriza la ""realidad"" del hid..."
2,2024-10-16,54.274.158,"En septiembre, el Presidente Gabriel Boric i...",Hidrógeno verde: La industria estratégica para...,https://www.emol.com/noticias/Economia/2024/10...,-,-,emol,chile,Hidrógeno verde: La industria estratégica para...
3,2024-10-14,54.293.366,– Eliana Ximena Oyarzún Nancuante La División ...,Subpesca solicita a la industria del hidrógeno...,https://laprensaaustral.cl/2024/10/15/subpesca...,-,-,laprensaaustral,chile,Subpesca solicita a la industria del hidrógeno...
4,2024-10-14,54.293.490,– Eliana Ximena Oyarzún Nancuante Su profunda ...,Sindicato de Trabajadores Enap llama a “aterri...,https://laprensaaustral.cl/2024/10/15/sindicat...,-,-,laprensaaustral,chile,Sindicato de Trabajadores Enap llama a “aterri...
...,...,...,...,...,...,...,...,...,...,...
411,2020-01-22,12.237.141,Hace décadas que el hidrógeno se ha promociona...,"Hidrógeno verde: Qué es, cómo se produce y cuá...",https://www.emol.com/noticias/Economia/2020/01...,"0,975",NEU,emol,chile,"Hidrógeno verde: Qué es, cómo se produce y cuá..."
412,2020-01-22,21.971.965,La crítica situación actual del cambio climáti...,Qué es el hidrógeno verde y porqué Chile podrí...,https://www.meganoticias.cl/nacional/289162-hi...,"0,993",NEU,ahoranoticiasmega,chile,Qué es el hidrógeno verde y porqué Chile podrí...
413,2020-01-21,12.237.509,Ante la necesidad de Chile -y el mundo- por ex...,"""Los números son extraordinarios para Chile"": ...",https://www.emol.com/noticias/Economia/2020/01...,"0,982",POS,emol,chile,"""Los números son extraordinarios para Chile"": ..."
414,2020-01-06,5.183.873,Los especialistas destacan que el hidrógeno re...,"Hidrógeno verde, la opción más eficaz para lim...",https://www.elciudadano.com/medio-ambiente/hid...,"0,997",POS,elciudadano,chile,"Hidrógeno verde, la opción más eficaz para lim..."


In [3]:
# Añadir noticias a ChromaDB sin proporcionar los embeddings
collection.upsert(
    documents=df['combined'].tolist(),
        metadatas=[{'media_outlet': row['media_outlet'], 'date': row['date'], 'title': row['title'], 'url': row['url']} for _, row in df.iterrows()],
    ids=[f'noticia_{i}' for i in range(len(df))]
)

# Función para realizar la búsqueda en ChromaDB
def search_in_chromadb(query):
    # Realiza la búsqueda en ChromaDB utilizando la consulta
    results = collection.query(
        query_embeddings=embeddings.embed_query(query),
        n_results=3  
    )
    return results

#Función que retorna los nombres junto cuantas veces aparece
def extraer_nombres_y_contar(texto):
    nombre_pattern = r"\b[A-Z][a-z]+\s[A-Z][a-z]+\b"  
    nombres = re.findall(nombre_pattern, texto)
    conteo = {}
    for nombre in nombres:
        conteo[nombre] = conteo.get(nombre, 0) + 1
    return conteo

In [4]:
from langchain.agents import Tool, initialize_agent
from langchain_openai import OpenAI
import openai
import requests
import re
from datetime import datetime
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.utilities import SerpAPIWrapper

from textblob import TextBlob  # Para análisis de sentimientos

serpapi = SerpAPIWrapper(serpapi_api_key="") #Pedir Serapi API key

# Función para búsqueda en ChromaDB (o cualquier otra base de datos)
def search_tool(query):
    search_results = search_in_chromadb(query)
    return "\n".join([str(doc) for doc in search_results['documents']])

# Función para búsqueda en Wikipedia
def wikipedia_search(query):
    response = requests.get(f"https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json()['query']['pages']
    page = next(iter(pages.values()))
    return page.get('extract', "No se encontró información en Wikipedia.")

def buscar_imagen(query):
    """
    Busca una imagen basada en el query proporcionado y devuelve el enlace de la imagen.
    """
    # Realizar la búsqueda con SerpAPI
    results = serpapi.run(f"{query} site:images.google.com")
    
    # Extraer el primer enlace de la imagen
    if results and "images_results" in results and len(results["images_results"]) > 0:
        return results["images_results"][0]["thumbnail"]
    return "No se encontró ninguna imagen para el query."

def contar_nombres_por_fechas(query):
    # Parsear el rango de fechas
    #match = re.search(r"desde: (\d{4}-\d{2}-\d{2}), hasta: (\d{4}-\d{2}-\d{2})", query)
    #if not match:
    #    return "Formato inválido. Usa 'desde: YYYY-MM-DD, hasta: YYYY-MM-DD'."
    
    fecha_inicio = datetime.strptime(match.group(1), "%Y-%m-%d").date()
    fecha_fin = datetime.strptime(match.group(2), "%Y-%m-%d").date()

    resultados = collection.query(query_texts=[consulta], n_results=10)
    
    for doc, metadata in zip(resultados['documents'], resultados['metadatas']):
        if 'date' in metadata:  # Verificar que exista el metadato 'date'
            doc_fecha = datetime.strptime(metadata['date'], "%Y-%m-%d").date()
            if fecha_inicio_dt <= doc_fecha <= fecha_fin_dt:
                documentos_filtrados.append({"contenido": doc, "metadata": metadata})
    
    if not docs:
        return "No se encontraron documentos dentro del rango de fechas."
    
    # Contar nombres en los documentos
    conteo_global = {}
    for doc in docs:
        conteo_doc = extraer_nombres_y_contar(doc.page_content)
        for nombre, count in conteo_doc.items():
            conteo_global[nombre] = conteo_global.get(nombre, 0) + count
    
    if not conteo_global:
        return "No se mencionaron nombres en los documentos encontrados."
    
    # Ordenar los nombres por número de menciones
    conteo_ordenado = sorted(conteo_global.items(), key=lambda x: x[1], reverse=True)
    return dict(conteo_ordenado)

def filtrar_noticias_por_persona(query):
    # Crear un patrón regex para el nombre (case-insensitive)
    patron = re.compile(rf'\b{re.escape(query)}\b', re.IGNORECASE)
    
    # Recuperar todos los documentos y metadatos de la colección
    all_docs = collection.get()["documents"]
    all_metadatas = collection.get()["metadatas"]
    
    # Filtrar los documentos que mencionen el nombre
    resultados = []
    for doc, metadata in zip(all_docs, all_metadatas):
        if patron.search(doc):  # Buscar en el contenido
            resultados.append({"titulo": metadata["titulo"], "contenido": doc})
    
    return resultados

# Inicializar el LLM
llm = OpenAI(temperature=0.7, openai_api_key=api_key)

# Crear herramientas
#tools = [
#    Tool(
#        name="ChromaDB Search",
#        func=search_tool,
#        description="Busca en la base de datos de ChromaDB los documentos relevantes."
#    ),
#    Tool(
#        name="Wikipedia Search",
#        func=wikipedia_search,
#        description="Consulta Wikipedia para obtener más información sobre el tema."
#    )
#]

tools = [
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    ),
    Tool(
        name="ChromaDB Search",
        func=search_tool,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Buscar Imagen",
        func=buscar_imagen,
        description="Busca imágenes relacionadas y devuelve el enlace de la imagen."
    ),
    Tool(
        name="Contar nombres por fechas",
        func=contar_nombres_por_fechas,
        description="Cuenta la cantidad de veces que aparece el nombre ne documento de determinadas fechas por tematica"
    ),
    Tool(
        name="Filtrar noticias por nombre de persona",
        func=filtrar_noticias_por_persona,
        description="Filtra las noticias por el nombre dado"
    )
]


# Definir el prompt usando PromptTemplate
prompt_template = """
Eres un agente que puede interactuar con herramientas para obtener información relevante para responder preguntas, que trabaja principalmente en español. 
Tienes acceso a 5 herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.
3. **Buscar Imagen**: Entrega el enlace de una imagen
4. **Contar nombres por fechas**: Devuelve la cantidad de menciones de nombres en documentos entre dos fechas por tematica
5. **Filtrar noticias por nombre de persona**: Devuelve las noticias que contienen el nombre de la persona.
Cuando recibas una pregunta, elige la herramienta que segun los siguientes formatos:

a. Si la pregunta busca la cantidad de menciones entre dos fechas por un tema, extrae la fecha de inicio,
fecha de fin y la tematica, para luego llamar a la funcion contar_nombres_por_fechas, dandole al entrada fecha_inicio, fecha_fin y tema.
, y devuelve el resultado de la funcion.
b. Si la pregunta busca quien es alguien o su opinion en algo, usa la funcion ChromaDB Search, y la funcion Wikipedia Search si fuese necesario	
c. si la pregunta busca la imagen o enlace de una imagen o foto, ocupa la funcion Buscar Imagen, dando el nombre como entrada
d. Si la pregunta busca las noticias que mencionan al nombre de una persona, ocupa la funcion filtrar_noticias_por_persona, usando el nombre como entrada o query.
Finalmente, con el output obtenida en la pregunta, formula una respuesta conscisa
"""

# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)

# Crear el agente React usando `initialize_agent` (asegurando que pasemos los argumentos correctamente)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\sebaa\AppData\Local\Temp\ipykernel_13960\3977409820.py:145: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [5]:
query = "¿Cuales son las personas que más se mencionan entre 2022/01/01 y 2023/01/01 sobre la temática del hidrogeno verde?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************1j8A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [16]:
query = "¿Quién es “Juan Carlos Jobet”?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [15]:
query = "¿Cuál es la opinión de Juan Carlos Jobet o su posición sobre el hidrógeno verde?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************R3YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [24]:
query = "¿Qué déclaraciones ha realizado Juan Carlos Jobet en el contexto del hidrógeno verde?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************R3YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [6]:
query = "Dame la imagen de Juan Carlos Jobet"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************R3YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [6]:
query = "¿Quienes son las personas más mencionadas en las noticias entre 2021/01/01 y 2022/01/01?"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************1j8A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}